In [1]:
import spacy
import pandas as pd
import multiprocessing
import numpy as np


In [3]:
df_hansard  = pd.read_csv('data/hansard_all.csv')
df_hansard.head()
# df_group = df_hansard.groupby('subjectOfBusinessId')


,Unnamed: 0,index,FloorLanguage,SubjectOfBusinessTitle,content,date,number,orderOfBusinessCatchLine,orderOfBusinessId,orderOfBusinessRubric,parliament,personId,personSpeaking,session,speakerName,subjectOfBusinessId,volume
0,0,0,EN,National Defence,"Mr. Speaker, last night, President Obama state...",2015-12-07,NUMBER 003,Oral Questions,8760272,OralQuestionPeriod,42nd PARLIAMENT,214611,"Hon. Rona Ambrose (Leader of the Opposition, CPC)",1st SESSION,Speaker: The Honourable Geoff Regan,8760273,VOLUME 148
1,1,1,EN,National Defence,"Mr. Speaker, Canada has spoken clearly in this...",2015-12-07,NUMBER 003,Oral Questions,8760272,OralQuestionPeriod,42nd PARLIAMENT,214296,"Right Hon. Justin Trudeau (Prime Minister, Lib.)",1st SESSION,Speaker: The Honourable Geoff Regan,8760273,VOLUME 148
2,2,2,EN,National Defence,"Mr. Speaker, let us just be clear about what I...",2015-12-07,NUMBER 003,Oral Questions,8760272,OralQuestionPeriod,42nd PARLIAMENT,214611,"Hon. Rona Ambrose (Leader of the Opposition, CPC)",1st SESSION,Speaker: The Honourable Geoff Regan,8760273,VOLUME 148
3,3,3,EN,National Defence,"Mr. Speaker, there is not a Canadian in this c...",2015-12-07,NUMBER 003,Oral Questions,8760272,OralQuestionPeriod,42nd PARLIAMENT,214296,"Right Hon. Justin Trudeau (Prime Minister, Lib.)",1st SESSION,Speaker: The Honourable Geoff Regan,8760273,VOLUME 148
4,4,4,EN,National Defence,"Mr. Speaker, last night President Obama also s...",2015-12-07,NUMBER 003,Oral Questions,8760272,OralQuestionPeriod,42nd PARLIAMENT,214611,"Hon. Rona Ambrose (Leader of the Opposition, CPC)",1st SESSION,Speaker: The Honourable Geoff Regan,8760273,VOLUME 148


In [33]:
# df_hansard  = pd.read_csv('data/hansard_all.csv')

# df_group = df_hansard.groupby('subjectOfBusinessId')

q_a = []
for i, index in df_group.groups.items():
    # don't bother with odd pairs
    if (len(index) % 2 != 0): 
        continue

    # Create conversatoin pairs
    t = df_hansard.iloc[list(index)]['content'].values
    
    temp = list(zip(t[::2], t[1::2], 
                    df_hansard.iloc[index]['SubjectOfBusinessTitle'],
                    df_hansard.iloc[index]['FloorLanguage'],
                    df_hansard.iloc[index]['date'],
                    df_hansard.iloc[index]['personSpeaking'],                
                   ))
#     temp.append(df_hansard.iloc[index]['SubjectOfBusinessTitle'])
#     print(list(zip(t[::2], t[1::2])))
#     print(df_hansard.iloc[list(index)]['SubjectOfBusinessTitle'])
#     print(temp)
    q_a.append(temp)
#     break

q_a = [item for sublist in q_a for item in sublist]

# print(q_a[0])
# q_a = q_a[:10000]

print('number of q & a', len(q_a))
    
df_q_a = pd.DataFrame(q_a)
df_q_a.columns = ['Q', 'A', 'SubjectOfBusinessTitle', 'FloorLanguage', 'date', 'personSpeaking']
df_q_a.to_csv('data/q_a_all.csv')
df_q_a.head()

number of q & a 27372


,Q,A,SubjectOfBusinessTitle,FloorLanguage,date,personSpeaking
0,"Mr. Speaker, our government believes any viola...","Mr. Speaker, I want to thank the member for Hu...",Veterans Affairs,EN,2012-05-08,"Mr. Ben Lobb (Huron—Bruce, CPC)"
1,"\n Mr. Speaker, Canadians want to...","\n Mr. Speaker, our government is...","Immigration, Refugees and Citizenship",EN,2017-02-24,Mr. Garnett Genuis (Sherwood Park—Fort Saskatc...
2,"\n Mr. Speaker, the Government of...","\n Mr. Speaker, our government ha...",Justice,EN,2017-05-04,"Mr. Luc Thériault (Montcalm, BQ)"
3,"\n Mr. Speaker, the Quebec justic...","\n Mr. Speaker, as I said, our go...",Justice,EN,2017-05-04,Mr. Marco Mendicino (Parliamentary Secretary t...
4,"Mr. Speaker, we only sign something if we agre...","Mr. Speaker, I understand the concerns raised ...",Foreign Investment,EN,2016-02-04,Mr. Alexandre Boulerice (Rosemont—La Petite-Pa...


In [34]:
# df_hansard  = pd.read_csv('data/hansard_all.csv')

# df_group = df_hansard.groupby('subjectOfBusinessId')

# q_a = []
# for i, index in df_group.groups.items():
#     # don't bother with odd pairs
#     if (len(index) % 2 != 0): 
#         continue

#     # Create conversatoin pairs
#     t = df_hansard.iloc[list(index)]['content'].values
#     q_a.append(list(zip(t[::2], t[1::2])))

# q_a = [item for sublist in q_a for item in sublist]

# # q_a = q_a[:10000]

# print('number of q & a', len(q_a))
    
# df_q_a = pd.DataFrame(q_a)
# df_q_a.columns = ['Q', 'A']
# df_q_a.to_csv('data/q_a_all.csv')
# df_q_a.tail()

In [39]:
# Load English tokenizer, tagger, parser, NER and word vectors
nlp = spacy.load('en')

def sentence_tokenizer(raw_text):
    # Create doc = nlp(raw_text) and parse sentences. 
    return u' '.join(['BOS ' + sent.string.strip() + ' EOS' for sent in nlp(raw_text).sents])


# print(list(nlp(df_q_a['Q'][2]).sents))
# print(df_q_a['Q'][2])
# print(sentence_tokenizer(df_q_a['Q'][2]))

In [36]:
%%time

def _apply_df(args):
    df, func, kwargs = args
    df['Q'] = df['Q'].apply(func, **kwargs)
    df['A'] = df['A'].apply(func, **kwargs)
    return df#df.apply(func, **kwargs)

def apply_by_multiprocessing(df, func, **kwargs):
    workers = kwargs.pop('workers')
    pool = multiprocessing.Pool(processes=workers)
    result = pool.map(_apply_df, [(d, func, kwargs) for d in np.array_split(df, workers)])
    pool.close()
    return pd.concat(list(result))
    



CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 4.77 µs


In [37]:
%%time

num_cores = multiprocessing.cpu_count()
print(num_cores) 
    
df_q_a = pd.read_csv('data/q_a_all.csv')

df_q_a = apply_by_multiprocessing(df_q_a, sentence_tokenizer,  workers=num_cores)
   
df_q_a.to_csv('data/q_a_all.csv')


# df_q_a = pd.read_csv('data/q_a_all.csv')
# df_q_a.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1, inplace=True)
# df_q_a.to_csv('data/q_a_all.csv')
# text= ' Mr. Speaker, when the Prime Minister interrupted a woman at a town hall, correcting her use of “mankind” with “peoplekind“, his mansplaining went viral. Around the world, the Prime Minister was mocked for his political correctness. The Prime Minister eventually conceded that it was a dumb joke, but his principal secretary, Gerald Butts, tweeted that any and all who criticized his boss were Nazis. The Prime Minister once said that any statement by Mr. Butts could be considered his own, and in this case?'
# print(df_q_a['Q'][10])
# print(sentence_t.okenizer(df_q_a['Q'][10]))

# df_q_a['Q'] = df_q_a['Q'].apply(sentence_tokenizer)
# df_q_a['A'] = df_q_a['A'].apply(sentence_tokenizer)
# df_q_a.to_csv('data/q_a.csv')
# df_q_a.tail()



8
CPU times: user 968 ms, sys: 132 ms, total: 1.1 s
Wall time: 42.8 s


In [38]:
df_q_a.head()

,Unnamed: 0,Q,A,SubjectOfBusinessTitle,FloorLanguage,date,personSpeaking
0,0,"BOS Mr. Speaker, our government believes any v...","BOS Mr. Speaker, I want to thank the member fo...",Veterans Affairs,EN,2012-05-08,"Mr. Ben Lobb (Huron—Bruce, CPC)"
1,1,"BOS Mr. Speaker, Canadians want to help victim...","BOS Mr. Speaker, our government is proud of Ca...","Immigration, Refugees and Citizenship",EN,2017-02-24,Mr. Garnett Genuis (Sherwood Park—Fort Saskatc...
2,2,"BOS Mr. Speaker, the Government of Quebec says...","BOS Mr. Speaker, our government has taken sign...",Justice,EN,2017-05-04,"Mr. Luc Thériault (Montcalm, BQ)"
3,3,"BOS Mr. Speaker, the Quebec justice minister h...","BOS Mr. Speaker, as I said, our government has...",Justice,EN,2017-05-04,Mr. Marco Mendicino (Parliamentary Secretary t...
4,4,"BOS Mr. Speaker, we only sign something if we ...","BOS Mr. Speaker, I understand the concerns rai...",Foreign Investment,EN,2016-02-04,Mr. Alexandre Boulerice (Rosemont—La Petite-Pa...


In [ ]:
df_q_a['Q_A'] = df_q_a[['Q', 'A']].apply(lambda x: u'\n'.join(x), axis=1)


print(len(df_q_a['Q_A']))
with open('data/Q_A_pairs.txt', 'w') as file:
    for row in df_q_a['Q_A']:
        file.write(row.lower().strip() + '\n')
